In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [16]:
'''
1. View the contents of the file hortonworks.txt . It represents semi-structured

content where the first string in each line is a Web page URL, followed by a comma-
separated list of terms that appear on that particular Web page.
'''

# set the path
file_path = "file:///home/talentum/test-jupyter/Evalution_Labs/hortonworks.txt"

# read path
fileRDD = sc.textFile(file_path)

# View the contents (first 10 lines)
first_lines = fileRDD.take(10)  

for line in first_lines:
  print(line)

http://hortonworks.com/,hadoop,webinars,articles,download,enterprise,team,reliability
http://hortonworks.com/products/,hortonworks,services,core,feed,deployments,board,required
http://hortonworks.com/products/hortonworksdataplatform/,apache,password,directors,enterprise
http://hortonworks.com/get-started/,data,downloads,founders,hdp,deployments
http://hortonworks.com/download/,register,hadoop,hdp,download,presentations,videos
http://hortonworks.com/community/,connect,password,download,articles,knowledgebase,hadoop
http://hortonworks.com/kb,platform,feed,core,hadoop
http://hortonworks.com/about-us/,about,hortonworks,apache,hadoop,founders,directors
http://hortonworks.com/about-us/contact-us/,contact,support,hortonworks,hdp,enterprise
http://hortonworks.com/resources/,hadoop,services,training,videos


In [17]:
'''
2. Write a Spark application using Rdd that takes a set of <key,value> pairs (key is Web
page URL and Value is list of terms on the web page) and inverts the index, so that
each value(term) becomes a key, and the list of Web page URLs where that term is
appearing as a list of values. You can see from the output file how this information
could be used as the beginnings of a web search engine.
'''

# split RDD lines into words
splitRDD = fileRDD.map(lambda line: line.split(','))

# creating flatmap so url is value and remaning words are keys
flatmapRDD = splitRDD.flatMap(lambda line: ((word, line[0]) for word in line[1:]))

# reduce by keys
finalRDD = flatmapRDD.reduceByKey(lambda url1, url2: url1 + ',' + url2)

# print the key, value
for key, value in finalRDD.collect():
    print(key, value)

hadoop http://hortonworks.com/,http://hortonworks.com/download/,http://hortonworks.com/community/,http://hortonworks.com/kb,http://hortonworks.com/about-us/,http://hortonworks.com/resources/,http://hortonworks.com/webinars/,http://hortonworks.com/resources/,http://hortonworks.com/hadoop-training/
webinars http://hortonworks.com/
enterprise http://hortonworks.com/,http://hortonworks.com/products/hortonworksdataplatform/,http://hortonworks.com/about-us/contact-us/
team http://hortonworks.com/
reliability http://hortonworks.com/,http://hortonworks.com/resources/
feed http://hortonworks.com/products/,http://hortonworks.com/kb,http://hortonworks.com/resources/
board http://hortonworks.com/products/
password http://hortonworks.com/products/hortonworksdataplatform/,http://hortonworks.com/community/
hdp http://hortonworks.com/get-started/,http://hortonworks.com/download/,http://hortonworks.com/about-us/contact-us/,http://hortonworks.com/events/,http://hortonworks.com/webinars/,http://hortonwor